In [ ]:
import numpy as np
import training_data_generation as tdg
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion();

# Generate and Save Training Data

In [ ]:
num_examples = 10
run_dir = "Data"

# Warning to save individual illumination data change "sys_params["run_clean"]" to False in "training_data_generation.py"
dataset, dataset_params, sys_params, facility_spec = tdg.main((_, run_dir, num_examples, random_seed))

# Diagnostic

In [ ]:
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import os
%matplotlib inline
np_complex = np.vectorize(complex)

In [ ]:
diag_dir = "Data"
iex = 0
with_pointing_markers = True
import_flipped = False
old_format = False
display_steradians = False
sys_params = tdg.define_system_params(diag_dir)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
print_list1 = []
print_list2 = []
print_list3 = []

## Analyse optimization outputs

In [ ]:
ind_profile = 0
dataset_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["dataset_params_filename"])
facility_spec = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["facility_spec_filename"])
dataset = nrw.read_general_netcdf(sys_params["root_dir"]+"/"+sys_params["trainingdata_filename"])

avg_flux = dataset["avg_flux"][iex, ind_profile]
real_modes = dataset["real_modes"][iex,ind_profile,:]
imag_modes = dataset["imag_modes"][iex,ind_profile,:]
rms = dataset["rms"][iex, ind_profile]

intensity_map_normalized = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
intensity_map_sr = (intensity_map_normalized+1)*avg_flux

if ind_profile == 0:
    if display_steradians:
        drive_map = intensity_map_sr
        drive_units = r"$\rm{W/sr}$"
    else:
        drive_map = (intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2
        drive_units = r"$\rm{W/cm^2}$"
else:
    drive_map = intensity_map_sr
    drive_units = r"$\rm{Mbar}$"

hp.mollview(drive_map, unit=drive_units,flip="geo")
hp.graticule()

complex_modes = np_complex(real_modes, imag_modes)
power_spectrum = uhp.alms2power_spectrum(complex_modes, dataset_params["LMAX"])

LMAX = dataset_params["LMAX"]
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/modes_"+str(iex) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
if with_pointing_markers:
    hp.mollview(drive_map, unit=drive_units,flip="geo")
    hp.graticule()
    
    deck_gen_params = nrw.read_general_netcdf(sys_params["root_dir"] + "/" + sys_params["deck_gen_params_filename"])

    print_list1, power_deposited = uim.readout_intensity(facility_spec, intensity_map_sr, use_ablation_pressure=ind_profile)
    print_list2 = uim.extract_run_parameters(iex, power_deposited, dataset_params, facility_spec, sys_params, deck_gen_params, use_ablation_pressure=ind_profile)
    print_list = print_list1 + print_list2
    stats_filename = sys_params["figure_location"]+"/stats"+str(iex)+".txt"
    uim.print_save_readout(print_list, stats_filename)
    
    port_theta = deck_gen_params["port_centre_theta"]
    port_phi = deck_gen_params["port_centre_phi"]
    
    pointing_theta = np.squeeze(deck_gen_params["theta_pointings"][iex,:])
    pointing_phi = np.squeeze(deck_gen_params["phi_pointings"][iex,:])

    num_ifriit_beams = int(facility_spec['nbeams'] / facility_spec['beams_per_ifriit_beam'])
    port_cartesian = np.zeros((num_ifriit_beams, 2))
    portc_cartesian = np.zeros((num_ifriit_beams, 2))
    pointing_cartesian = np.zeros((num_ifriit_beams, 2))
    
    for ibeam in range(0, num_ifriit_beams):
        port_cartesian[ibeam,0], port_cartesian[ibeam,1] = uim.angle2moll(facility_spec["Theta"][ibeam], facility_spec["Phi"][ibeam])
        portc_cartesian[ibeam,0], portc_cartesian[ibeam,1] = uim.angle2moll(deck_gen_params["port_centre_theta"][ibeam], deck_gen_params["port_centre_phi"][ibeam])
        pointing_cartesian[ibeam,0], pointing_cartesian[ibeam,1] = uim.angle2moll(pointing_theta[ibeam], pointing_phi[ibeam])
        dx = pointing_cartesian[ibeam,0] - portc_cartesian[ibeam,0]
        dy = pointing_cartesian[ibeam,1] - portc_cartesian[ibeam,1]
        # reduce line length to give space for arrow head
        head_length = 0.05
        shaft_angle = np.arctan2(dy, dx)
        shaft_length = np.sqrt(dx**2 + dy**2) - head_length
        if shaft_length > 0.0:
            dx = shaft_length * np.cos(shaft_angle)
            dy = shaft_length * np.sin(shaft_angle)
            
            if np.abs(dx) < 1.0: # don't plot arrow for pointings which cross the edge
                plt.arrow(portc_cartesian[ibeam,0], portc_cartesian[ibeam,1], dx, dy, linewidth=3, head_width=head_length, head_length=head_length, fc='k', ec='k')
        else:
            plt.arrow(portc_cartesian[ibeam,0], portc_cartesian[ibeam,1], dx, dy)

    plt.scatter(port_cartesian[:,0], port_cartesian[:,1],c="white", marker="s")
    plt.scatter(pointing_cartesian[:,0], pointing_cartesian[:,1],c="white", marker="x")

    plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+str(iex) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Analysis from ifriit output

In [ ]:
num_examples = 1
sys_params = tdg.define_system_params(diag_dir)
sys_params["run_gen_deck"] = False

if old_format:
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
else:
    dataset_params = nrw.read_general_netcdf(diag_dir + "/" + sys_params["dataset_params_filename"])
    facility_spec = nrw.read_general_netcdf(diag_dir + "/" + sys_params["facility_spec_filename"])
dataset_params["num_examples"] = num_examples

In [ ]:
if old_format:
    intensity_map_sr = nrw.read_intensity(run_location, dataset_params["imap_nside"], facility_spec['target_radius'])
    # convert from W/sr to W/cm**2
    intensity_map = intensity_map_sr / (facility_spec['target_radius'] / 10000.0)**2
else:
    parameters = nrw.read_general_netcdf(run_location + "/" + sys_params["ifriit_ouput_name"])
    intensity_map = parameters["intensity"]
    # convert from W/cm**2 to W/sr
    intensity_map_sr = parameters["intensity"] * (facility_spec['target_radius'] / 10000.0)**2

intensity_map_normalized, avg_flux = uim.imap_norm(intensity_map)
rms = uim.extract_rms(intensity_map_normalized)

print_list1, power_deposited = uim.readout_intensity(facility_spec, intensity_map_sr)
stats_filename = run_location+"/stats"+str(iex)+".txt"
print_list = print_list1 + print_list2 + print_list3
uim.print_save_readout(print_list1, stats_filename)

if display_steradians:
    hp.mollview(intensity_map_sr, unit=r"$\rm{W/sr}$", flip="geo")
else:
    hp.mollview(intensity_map, unit=r"$\rm{W/cm^2}$", flip="geo")
hp.graticule()
plt.savefig(run_location+"/init_illumination" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted = uhp.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/init_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
hp.mollview((intensity_map_normalized+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

real_modes, imag_modes = uhp.imap2modes(intensity_map_normalized, LMAX)
intensity_map_normalized2 = uhp.modes2imap(real_modes, imag_modes, dataset_params["imap_nside"])
hp.mollview((intensity_map_normalized2+1)*avg_power, unit=r"$\rm{W/cm^2}$", flip="geo")

In [ ]:
intensity_map_normalized, avg_power = uim.imap_norm(intensity_map)
intensity_map_init = (intensity_map_normalized+1) * avg_power
press_abla = 24.7 * (intensity_map_init / 1.0e14)**(2.0/3.0)
hp.mollview(press_abla, unit=r"$\rm{Mbars}$", flip="geo")
press_abla_mean = np.mean(press_abla)
init_press_abla_rms = np.sqrt(np.mean((press_abla / press_abla_mean - 1.0)**2)) *100.0
print('Initial ablation pressure RMS: {:.2f}%, '.format(init_press_abla_rms))
print('Initial mean ablation pressure: {:.2f}Mbar, '.format(press_abla_mean))

## Open Ifriit heat source modes

In [ ]:
file_name = sys_params["heat_source_nc"]
hs_and_modes = nrw.read_general_netcdf(run_location+"/"+file_name)

if old_format:
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
else:
    dataset_params = nrw.read_general_netcdf(diag_dir + "/" + sys_params["dataset_params_filename"])
    facility_spec = nrw.read_general_netcdf(diag_dir + "/" + sys_params["facility_spec_filename"])


In [ ]:
avg_flux = hs_and_modes["average_flux"][0]
#print('Mean ablation pressure: {:.2f}Mbar, '.format(avg_flux))

lmax = 30
np_complex = np.vectorize(complex)
X_train_complex = np_complex(hs_and_modes["complex_modes"][0,:], hs_and_modes["complex_modes"][1,:])

intensity_map_normalized = hp.alm2map(X_train_complex, lmax)
#hp.mollview(intensity_map_normalized, unit=r"$\rm{W/sr}$",flip="geo")
hp.mollview((intensity_map_normalized+1)*avg_flux, unit=r"$\rm{Mbar}$",flip="geo")
hp.graticule()
plt.savefig(run_location+"/ablation_pressure" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

print_list2, power_deposited = uim.readout_intensity(facility_spec, (intensity_map_normalized+1)*avg_flux, use_ablation_pressure=1)
print_list = print_list1 + print_list2 + print_list3
stats_filename = run_location+"/stats"+str(iex)+".txt"
uim.print_save_readout(print_list, stats_filename)

#hp.mollview((intensity_map_normalized+1)*avg_flux / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")

power_spectrum = uhp.alms2power_spectrum(X_train_complex, lmax)
#print("The rms direct from output modes: ", np.sqrt(np.sum(power_spectrum))*100.0, "%")

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(lmax), np.sqrt(power_spectrum) * 100.0)
ax.set_xticks(range(0, lmax+1, int(lmax/5)))
plt.xlim([0, lmax])
plt.title("Pressure Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/ablation_pressure_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
theta_edges = (hs_and_modes["theta"][1:] + hs_and_modes["theta"][:-1]) / 2.0
theta_edges = np.append(0.0, theta_edges)
theta_edges = np.append(theta_edges, np.pi)
phi_edges = (hs_and_modes["phi"][1:] + hs_and_modes["phi"][:-1]) / 2.0
phi_edges = np.append(0.0, phi_edges)
phi_edges = np.append(phi_edges, 2.0*np.pi)

theta_grid, phi_grid = np.meshgrid(theta_edges, phi_edges)

dphi = phi_grid[1:,1:] - phi_grid[:-1,1:]
d_cos_theta = np.cos(theta_grid[1:,1:]) - np.cos(theta_grid[1:,:-1])
domega = np.abs(dphi * d_cos_theta)

total_pwr = np.sum(hs_and_modes["heat_source"] * domega)
surface_area = 4.0 * np.pi * (facility_spec['target_radius'] / 10000.0)**2
avg_flux = total_pwr / (4.0 * np.pi)

print('Mean intensity is {:.2e}W/cm^2, '.format(total_pwr / surface_area)) #/ 4.0 / np.pi
print('Intensity per steradian, {:.2e}W/sr'.format(avg_flux))

In [ ]:
theta_grid, phi_grid = np.meshgrid(hs_and_modes["theta"], hs_and_modes["phi"])
plt.pcolor(theta_grid, phi_grid, hs_and_modes["heat_source"])
plt.colorbar()

nside_low = 16
nside = 32

pixel_ind = hp.ang2pix(nside_low, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_low = np.zeros(hp.nside2npix(nside_low))
imap_low[pixel_ind] = hs_and_modes["heat_source"].reshape(-1)
imap_upgrade = hp.ud_grade(imap_low, nside)

pix_ind = hp.ang2pix(nside, theta_grid.reshape(-1), phi_grid.reshape(-1))
imap_high = np.zeros(hp.nside2npix(nside))
imap_upgrade[pix_ind] = hs_and_modes["heat_source"].reshape(-1)

# convert from W/sr to W/cm**2  
hp.mollview(imap_upgrade / (facility_spec['target_radius'] / 10000.0)**2, unit=r"$\rm{W/cm^2}$",flip="geo")

avg_flux = np.mean(imap_upgrade) / (facility_spec['target_radius'] / 10000.0)**2

hp.graticule()
plt.savefig(run_location+"/heatsource" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

print_list3, power_deposited = uim.readout_intensity(facility_spec, imap_upgrade)
print_list = print_list1 + print_list2 + print_list3
stats_filename = run_location+"/stats"+str(iex)+".txt"
uim.print_save_readout(print_list, stats_filename)

LMAX = 30
power_spectrum_unweighted = uhp.power_spectrum(imap_upgrade, LMAX)

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, LMAX])
plt.title("Intensity Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(run_location+"/hs_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Check aster input netcdf

In [ ]:
file_name = "ifriit_1davg_input.nc" # "ifriit_1davg_input.nc"#"N210519" #"N190204-003"
aster_input = nrw.read_general_netcdf(run_location+"/"+file_name)
print(aster_input.keys())

"""
iex2 = iex #change comparison file here
run_loc2 = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex2) 
file_name = "ifriit_input_chic.nc" #"N190204-003"
aster_input2 = nrw.read_general_netcdf(run_loc2+"/"+file_name)
print(aster_input2.keys())

for key in aster_input.keys():
    print(key)
    print(np.shape(aster_input[key]))
    #print(np.shape(aster_input2[key]))
#"""

plt.figure()
var = "ne"

plt.semilogy(aster_input['xs'].reshape(-1),aster_input[var].reshape(-1),label="Aster, N21")
#plt.semilogy(aster_input2['xs'][:],aster_input2[var][0,:],label="Chic, foam target")
plt.xlabel(r"Radius ($\mu m$)")
plt.ylabel(var)
#plt.xlim([0, 6000])
#plt.ylim([25, 30])
plt.legend()
plt.savefig(run_location+"/profiles_" + var + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


In [ ]:
file_name = "kHiper.nc" #"N190204-003"
chic_output = nrw.read_general_netcdf(run_location+"/"+file_name)
print(chic_output.keys())

time = 1.0e-9
time_ind = np.argmin(np.abs(chic_output["temps"]-time))
print(time_ind, chic_output["temps"][time_ind])

var = "ne"
print(np.shape(chic_output["xc"][time_ind]))
print(np.shape(chic_output[var][time_ind]))
plt.plot(chic_output["xc"][time_ind],chic_output[var][time_ind])

## Analyse aster (/dump32) output netcdf

In [ ]:
from netCDF4 import Dataset

In [ ]:
def read_general_netcdf_test(filename):
    parameters = {}

    rootgrp = Dataset(filename)
     
    grp = rootgrp["data"]
    keys = list(grp.variables.keys())
    for key in keys:
        if np.shape(np.shape(grp[key]))[0] == 2:
            parameters[key] = grp[key][:,:]
        if np.shape(np.shape(grp[key]))[0] == 1:
            parameters[key] = grp[key][:]
    rootgrp.close()

    return parameters

In [ ]:
sys_params = tdg.define_system_params(diag_dir)
file_name = "dump31_1davg.nc"

input_dir = run_location + "/" + file_name
aster_output0 = read_general_netcdf_test(input_dir)

for key in aster_output0.keys():
    print(key, np.shape(aster_output0[key]))

lmax = 30

time_step = 50

label_0 = "run_" + str(folder_num) + ", time {:.2f}ns".format(aster_output0["time"][time_step])
shell_thickness = aster_output0["sph"][time_step,0]
aster_output0["sph"][time_step,0] = 0.0
plt.plot(aster_output0["sph"][time_step,:lmax]/shell_thickness*100, label=label_0)
rms = np.sqrt(np.sum((aster_output0["sph"][time_step,1:lmax]/shell_thickness)**2))*100.0
print("The RMS for " + label_0 + " is: ", rms, "%")

"""
folder_num = 1
input_dir = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(folder_num) + "/" + file_name
aster_output1 = read_general_netcdf_test(input_dir)

time_step = time_step + 0 #for time offset

label_1 = "run_" + str(folder_num) + ", time {:.2f}ns".format(aster_output1["time"][time_step])
shell_thickness = aster_output1["sph"][time_step,0]
aster_output1["sph"][time_step,0] = 0.0
plt.plot(aster_output1["sph"][time_step,:lmax]/shell_thickness*100, label=label_1)
rms = np.sqrt(np.sum((aster_output1["sph"][time_step,1:lmax]/shell_thickness)**2))*100.0
print("The RMS for " + label_1 + " is: ", rms, "%")
#"""

plt.title("Spherical harmonics for areal density")
plt.ylabel("Deviation from mean (%)")
plt.xlabel("Mode")
plt.legend()
plt.savefig(run_location+"/Comparison_spherical_modes_areal_density" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

# Combine Datasets

In [ ]:
from netCDF4 import Dataset
from os import path
import os
import numpy as np
import netcdf_read_write as nrw
sys_params = tdg.define_system_params(run_dir)

In [ ]:
dataset1 = "Data1"
dataset2 = "Data2"

new_file = "Data3"

filename_trainingdata = dataset1 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_1 = training_data.variables["X_train"][:]
Y_1 = training_data.variables["Y_train"][:]
avg_powers_1 = training_data.variables["avg_powers"][:]
training_data.close()

filename_trainingdata = dataset2 + "/" + sys_params["trainingdata_filename"] 
training_data = Dataset(filename_trainingdata)
X_2 = training_data.variables["X_train"][:]
Y_2 = training_data.variables["Y_train"][:]
avg_powers_2 = training_data.variables["avg_powers"][:]
training_data.close()

X_train = np.hstack((X_1, X_2))
Y_train = np.hstack((Y_1, Y_2))
avg_powers = np.hstack((avg_powers_1, avg_powers_2))
print(np.shape(X_train), np.shape(Y_train), np.shape(avg_powers))

filename_trainingdata = new_file + "/" + sys_params["trainingdata_filename"] 
nrw.save_training_data(X_train, Y_train, avg_powers, filename_trainingdata)

# Create time varying input deck

In [ ]:
import csv
import utils_intensity_map as uim
import netcdf_read_write as nrw
import utils_deck_generation as idg
import os

In [ ]:
load_from_file = False

iex = 0
run_dir = "Data"
num_examples = 0
sys_params = tdg.define_system_params(run_dir)

if load_from_file:
    dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
    #print(deck_gen_params["p0"][iex,:])
    #print(dataset_params["num_variables_per_beam"])
    power_slice = slice(dataset_params["num_variables_per_beam"]-1, dataset_params["num_input_params"], dataset_params["num_variables_per_beam"])
    #print(power_slice)
    cone_multiplier = deck_gen_params["sim_params"][iex,power_slice]
    print(cone_multiplier)
    #print(deck_gen_params["sim_params"][iex,:])
else:
    dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
    cone_multiplier = np.zeros(dataset_params["num_variables_per_beam"]) + 1.0

num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere


## Read from "requested power" csv (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N190204-003"
file_name = "N190204-003-999_LEH_requested_beam_powers"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

beam_indices = [0] * facility_spec['nbeams']
beam_names = [0] * facility_spec['nbeams']
times = np.zeros((num_lines-1))
powers = np.zeros((facility_spec['nbeams'],num_lines-1))

total_power = 0

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)

backlighter_beams = ("B161","B162","B163","B164","B415","B416","B417","B418")

irow=0
for row in pulse_data:
    if irow == 0:
        nbeams = len(row) - 1
        for ibeam in range(0,nbeams):
            #print(row[ibeam][2:])
            beam_names[ibeam] = "B"+row[ibeam+1][2:]
            beam_indices[ibeam] = np.where(facility_spec['Beam']==beam_names[ibeam])[0][0]
    else:
        times[irow-1] = row[0]
        powers[0:,irow-1] = row[1:]
    irow += 1
pulse_file.close()

backlighter_modified_beams = []
idealized_total_power_profile = 0
total_power_no_backlighter = 0

fig = plt.figure()
ax = plt.axes()
for ibeam in range(0,nbeams):
    if beam_names[ibeam] in backlighter_beams:
        print("Beam: " + beam_names[ibeam] + " is for the backlighter")
    else:
        total_power_no_backlighter += powers[ibeam,:]
        plt.plot(times, powers[ibeam,:])
    if np.max(powers[ibeam,:]) > 1.0:
        backlighter_modified_beams.append(beam_names[ibeam])
print(backlighter_modified_beams)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_power_with_backlighter = np.sum(powers, axis=0)
total_energy_with_backlighter = np.trapz(total_power_with_backlighter, x=times)
print("Total energy requested (with backlighter): {:.2f}".format(total_energy_with_backlighter), "kJ")
print("Max power (with backlighter): {:.2f}".format(np.max(total_power_with_backlighter)), "TW")

total_power = total_power_no_backlighter
total_energy = np.trapz(total_power_no_backlighter, x=times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times, total_power_no_backlighter)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

idealized_power_profile = total_power / nbeams

fig = plt.figure()
ax = plt.axes()
plt.plot(times, idealized_power_profile)
plt.xlim([-0.5, 5.0])
plt.xlabel("Time (ns)")
plt.ylabel("Idealized Power Profile per beam, no backlighter (TW)")
plt.savefig(file_dirname+'/idealized_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

times_old = times
powers_old = powers
total_power_old = total_power
times = np.zeros((num_cones,num_lines-1))
powers = np.zeros((num_cones,num_lines-1))
total_power = np.zeros((num_lines-1))

for icone in range(0,num_cones):
    powers[icone,:] = idealized_power_profile
    times[icone,:] = times_old
    total_power += powers[icone,:]

## Read from "Ring" csv (per beam)

In [ ]:
file_dirname = "laser_pulse_data_pdd/N210519"
file_name = "I_PDD_BePDD_DDShap_EEE_Ring"
file_type = ".csv"

num_lines = sum(1 for line in open(file_dirname + "/" + file_name + str(1) + file_type))

times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
fig = plt.figure()
ax = plt.axes()

total_power = 0
for icone in range(num_cones):
    pulse_file = open(file_dirname + "/" + file_name + str(icone+1) + file_type)
    pulse_data = csv.reader(pulse_file)
    i = 0
    for row in pulse_data:
        times[icone,i] = float(row[0])
        powers[icone,i] = float(row[1])
        i += 1

    pulse_file.close()
    
    total_power += powers[icone,:] * facility_spec["beams_per_cone"][icone] * 2
    
    plt.plot(times[icone,:], powers[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy = np.trapz(total_power, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
print("Max power: {:.2f}".format(np.max(total_power)), "TW")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Read from Olson paper data extraction (all beam pulse)

In [ ]:
file_dirname = "laser_pulse_data_pdd/ABLE_campaign"
file_name = "PDD_foam_target_olson_paper_pulse_data_extraction"
file_type = ".csv"


num_lines = sum(1 for line in open(file_dirname + "/" + file_name + file_type))

total_power_times = np.zeros((num_lines))
total_power = np.zeros((num_lines))

pulse_file = open(file_dirname + "/" + file_name + file_type)
pulse_data = csv.reader(pulse_file)
i = 0
for row in pulse_data:
    total_power_times[i] = float(row[0])
    total_power[i] = float(row[1])
    i += 1

pulse_file.close()

total_energy = np.trapz(total_power, x=total_power_times)
print("Total energy requested: {:.2f}".format(total_energy), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64

fig = plt.figure()
ax = plt.axes()
plt.plot(total_power_times, total_power)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
plt.savefig(file_dirname+'/laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
times = np.zeros((num_cones,num_lines))
powers = np.zeros((num_cones,num_lines))
for icone in range(num_cones):
    times[icone,:] = total_power_times
    powers[icone,:] = total_power / facility_spec["nbeams"]

# Add extra 8% power for 23.5 cones
powers[0,:] = powers[0,:] * 1.08
renorm_fac = (facility_spec["nbeams"] - facility_spec["beams_per_cone"][0] * 2 
              + facility_spec["beams_per_cone"][0] * 2 * 1.08) / 192
powers = powers / renorm_fac

fig = plt.figure()
ax = plt.axes()
total_energy = 0.0
for icone in range(num_cones):
    plt.plot(times[icone,:], powers[icone,:], label=facility_spec['cone_names'][icone])
    cone_powers = powers[icone,:] * facility_spec["beams_per_cone"][icone] * 2
    cone_energy = np.trapz(cone_powers, x=times[icone,:])
    print(cone_energy)
    total_energy += cone_energy
print(total_energy)
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
plt.savefig(file_dirname+'/laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Output pulse shapes unmodified

In [ ]:


for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers[icone,irow]))
            out.write('\r\n')
    out.close()


## Output pulse shapes with cone power fractions

In [ ]:
#print(facility_spec['beams_per_cone'])
#for icone in range(facility_spec['num_cones']):

In [ ]:
normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
powers_reweighted = np.zeros((np.shape(times)))

for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    powers_reweighted[icone,:] = normalized_power[:] * cone_multiplier[icone]
    total_power_reweighted += powers_reweighted[icone,:]
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
#plt.xlim([-0.5, 10.0])
plt.legend()
plt.savefig(file_dirname+'/new_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
#plt.xlim([-0.5, 10.0])
plt.savefig(file_dirname+'/new_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Rescale powers for same total energy

In [ ]:
scaling = total_energy / total_energy_reweighted
print(scaling)

normalized_power = total_power / np.max(total_power)
total_power_reweighted = 0.0
powers_reweighted = np.zeros((np.shape(times)))

for ibeam in range(facility_spec['nbeams']):
    beam = facility_spec['Beam'][ibeam]
    file_name = file_dirname + "/pulse_" + beam + ".txt"
    
    cone_name = facility_spec["Cone"][ibeam]
    if (cone_name == 23.5):
        icone = 0
    elif (cone_name == 30):
        icone = 1
    elif (cone_name == 44.5):
        icone = 2
    else:
        icone = 3
    powers_reweighted[icone,:] = normalized_power[:] * cone_multiplier[icone] * scaling
    total_power_reweighted += powers_reweighted[icone,:]
    
    with open(file_name,"w") as out:
        for irow in range(len(times[0,:])):
            out.write(str(times[icone,irow]) + ' ' + str(powers_reweighted[icone,irow]))
            out.write('\r\n')
    out.close()

print(np.max(powers_reweighted, axis=1))
fig = plt.figure()
ax = plt.axes()
for icone in range(num_cones):
    plt.plot(times[icone,:], powers_reweighted[icone,:], label=facility_spec['cone_names'][icone])
plt.xlabel("Time (ns)")
plt.ylabel("Power per beam (TW)")
plt.legend()
#plt.xlim([-0.5, 10.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_per_beam' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

total_energy_reweighted = np.trapz(total_power_reweighted, x=times[0,:])
print("Total energy requested: {:.2f}".format(total_energy_reweighted), "kJ")
#total_power = powers[0,:] * 32 + powers[1,:] * 32 + powers[2,:] * 64 + powers[3,:] * 64
print("Max power: {:.2f}".format(np.max(total_power_reweighted)), "TW")

fig = plt.figure()
ax = plt.axes()
plt.plot(times[0,:], total_power_reweighted)
plt.xlabel("Time (ns)")
plt.ylabel("Total Power (TW)")
#plt.xlim([-0.5, 10.0])
plt.savefig(file_dirname+'/new_scaled_laser_pulse_shape_total' + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')


## Output decks with beam pointings

In [ ]:
#sys_params = tdg.define_system_params(file_dirname)
#sys_params["run_clean"] = False
laser_init_time = 0.0
print("Lasers set to initialize at time: {:.2f}ns".format(laser_init_time))
deck_gen_params['t0'] = laser_init_time

#dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

#idg.generate_input_deck(facility_spec, file_dirname)

if os.path.exists(file_dirname + "/ifriit_inputs.txt"):
    os.remove(file_dirname + "/ifriit_inputs.txt")
idg.generate_input_pointing_and_pulses(iex, facility_spec, deck_gen_params, file_dirname, dataset_params["run_type"])

# Beam Spot

In [ ]:
def nif_beam_spot(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    c1 = sup_gauss_dict["c1"]
    c2 = sup_gauss_dict["c2"]
    c3 = sup_gauss_dict["c3"]
    intensity = c0 * np.exp(-((x/c1)**2+(y/c2)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3 = sup_gauss_dict["c3"]
    c5 = 2.0**c3 * np.log(2.0)
    intensity = c0 * np.exp(-c5*((x/FWHMx)**2+(y/FWHMy)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM_mike(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3 = sup_gauss_dict["c3"]
    c5 = 2.0 * np.log(2.0)**c3
    intensity = np.exp(-((-x/FWHMx)**2+(-y/FWHMy)**2)**(c3/2))
    
    return intensity

In [ ]:
def nif_beam_spot_FWHM_xavier(x, y, sup_gauss_dict):
    
    c0 = sup_gauss_dict["c0"]
    FWHMx = sup_gauss_dict["FWHMx"]
    FWHMy = sup_gauss_dict["FWHMy"]
    c3x = sup_gauss_dict["c3x"]
    c3y = sup_gauss_dict["c3y"]
    c5 = 2.0 * np.log(2.0)**c3
    
    dr = np.sqrt(x**2 + y**2)
    alpha = np.sqrt((c3x * x / dr)**2 + (c3y * y / dr)**2)
    intensity = np.exp(-((-x / FWHMx)**2 + (-y / FWHMy)**2)**(alpha / 2))
    
    return intensity

In [ ]:
def super_gaussian_shape(c0, c1, c2, c3):
    
    sup_gauss_dict = {}
    sup_gauss_dict["c0"] = c0
    sup_gauss_dict["c1"] = c1
    sup_gauss_dict["c2"] = c2
    sup_gauss_dict["c3"] = c3
    
    return sup_gauss_dict

In [ ]:
run_dir = "Data"
num_examples = 1
sys_params = tdg.define_system_params(run_dir)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
num_cones = int(facility_spec['num_cones']/2) # Symmetry in top bottom hemisphere

## All NIF CPP

In [ ]:
mesh_lim = 1.5
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

c0 = [0.55273, 0.62017, 1.30020, 1.46540]
c1 = [0.93836, 0.89096, 0.67533, 0.65491]
c2 = [0.66859, 0.63113, 0.40627, 0.37022]
c3 = [8.7514,  7.8105,  4.3210,  4.3427]

r_a = [0.882, 0.824, 0.635, 0.593]
r_b = [0.631, 0.590, 0.367, 0.343]

fig = plt.figure()
for icone in range(0, num_cones):
    sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
    intensity = nif_beam_spot(X, Y, sup_gauss_dict)
    fig.add_subplot(1, 4, icone+1)
    plt.pcolormesh(X, Y, intensity)
    plt.gca().set_aspect('equal', adjustable='box')
    
    # Tom Chapman uses a r_a threshold in his data
    #intensity_threshold = nif_beam_spot(r_a[icone], 0.0, sup_gauss_dict)
    # This is a more correct way to do the Chapman threshold
    #intensity_threshold = nif_beam_spot(r_a[icone]/np.sqrt(2), r_b[icone]/np.sqrt(2), sup_gauss_dict)
    # the 1/e
    intensity_threshold = nif_beam_spot(c1[icone]/np.sqrt(2), c2[icone]/np.sqrt(2), sup_gauss_dict)
    
    print(intensity_threshold, np.max(intensity), c0[icone], np.exp(-1) * c0[icone])

    inside_indices = np.where(intensity > intensity_threshold)
    
    power_inside = np.sum(intensity[inside_indices])
    power_total = np.sum(intensity)
    power_fraction = power_inside / power_total
    print("Ring ", icone)
    print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
    print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
    print("Area of a.b ellipse {:.3f}, ".format(np.pi * r_a[icone] * r_b[icone]))
    print("Area of c1.c2 ellipse {:.3f}, ".format(np.pi * c1[icone] * c2[icone]))


In [ ]:
FWHMx = [1.800, 1.700, 1.241, 1.204]
FWHMy = [1.282, 1.204, 0.7465, 0.6805]

fig = plt.figure()
for icone in range(0, num_cones):
    sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
    sup_gauss_dict["FWHMx"] = FWHMx[icone]
    sup_gauss_dict["FWHMy"] = FWHMy[icone]
    intensity = nif_beam_spot_FWHM(X, Y, sup_gauss_dict)
    
    fig.add_subplot(1, 4, icone+1)
    plt.pcolormesh(X, Y, intensity)
    plt.gca().set_aspect('equal', adjustable='box')
    
    #intensity_threshold = nif_beam_spot_FWHM(FWHMx[icone]/2.0, 0.0, sup_gauss_dict)
    intensity_threshold = nif_beam_spot_FWHM(FWHMx[icone]/2.0/np.sqrt(2), FWHMy[icone]/2.0/np.sqrt(2), sup_gauss_dict)
    #intensity_threshold = nif_beam_spot_FWHM(2.0*FWHMx[icone]*np.log(2.0)**(1.0/c3[icone]), 0.0, sup_gauss_dict)
    inside_indices = np.where(intensity > intensity_threshold)
    
    power_inside = np.sum(intensity[inside_indices])
    power_total = np.sum(intensity)
    power_fraction = power_inside / power_total
    print("Ring ", icone)
    print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
    print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
    print("Area of FWHMx/2.FWHMy/2 ellipse {:.3f}, ".format(np.pi * FWHMx[icone]/2.0 * FWHMy[icone]/2.0))
    

## 23.5 deg CPP 

In [ ]:

x = np.linspace(-2,2,1000)
y = np.linspace(-2,2,1000)
X, Y = np.meshgrid(x,y)

c0 = 0.55273
c1 = 0.93836
c2 = 0.66859
c3 = 8.7514

r_a = 0.882
r_b = 0.631

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')

intensity_threshold = nif_beam_spot(c1, 0.0, sup_gauss_dict)
intensity_threshold = nif_beam_spot(c1/np.sqrt(2), c2/np.sqrt(2), sup_gauss_dict)
intensity_threshold = nif_beam_spot(r_a, 0.0, sup_gauss_dict)
print(intensity_threshold)
print(nif_beam_spot(0.0, c2, sup_gauss_dict))

inside_indices = np.where(intensity > intensity_threshold)
print(np.shape(inside_indices))
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print(power_fraction)

intensity[inside_indices] = 0.0
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.gca().set_aspect('equal', adjustable='box')


## LMJ-A CPP 

In [ ]:
mesh_lim = 1.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([2.4])  # Guess

r_a = np.array([0.435]) # Guess
r_b = np.array([0.225])  # Guess

icone = 0

sup_gauss_dict = super_gaussian_shape(c0, c1, c2, c3)
intensity = nif_beam_spot(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_old_def = intensity

intensity = nif_beam_spot(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
intensity = nif_beam_spot(X, Y, sup_gauss_dict)
    
intensity_threshold = nif_beam_spot(c1[icone]/np.sqrt(2), c2[icone]/np.sqrt(2), sup_gauss_dict)
    
print(intensity_threshold, np.max(intensity), c0[icone], np.exp(-1) * c0[icone])
intensity_threshold = 0.13 * np.max(intensity)

inside_indices = np.where(intensity > intensity_threshold)
    
power_inside = np.sum(intensity[inside_indices])
power_total = np.sum(intensity)
power_fraction = power_inside / power_total
print("Power fraction enclosed by threshold {:.3f}, ".format(power_fraction))
print("Area of within threshold {:.3f}, ".format(np.shape(inside_indices)[1] * (2.0 * mesh_lim / num_points)**2))
print("Area of a.b ellipse {:.3f}, ".format(np.pi * r_a[icone] * r_b[icone]))
print("Area of c1.c2 ellipse {:.3f}, ".format(np.pi * c1[icone] * c2[icone]))


In [ ]:
# Mike Belyaev's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([2.7])

FWHMx = [0.435]
FWHMy = [0.225]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]

mesh_lim = 1.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity = nif_beam_spot_FWHM_mike(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_single_order_x = intensity

intensity = nif_beam_spot_FWHM_mike(0.0, y, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')
intensity_single_order_y = intensity

intensity = nif_beam_spot_FWHM_mike(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

In [ ]:
# Xavier's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.435]) # Guess
c2 = np.array([0.225]) # Guess
c3 = np.array([0.0])
c3x = np.array([3.0])
c3y = np.array([2.4])

FWHMx = [0.435]
FWHMy = [0.225]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]
sup_gauss_dict["c3x"] = c3x[icone]
sup_gauss_dict["c3y"] = c3y[icone]

mesh_lim = 1.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity = nif_beam_spot_FWHM_xavier(x, 0.0, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.plot(x,intensity_single_order_x)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')

intensity = nif_beam_spot_FWHM_xavier(0.0, y, sup_gauss_dict)
plt.figure()
plt.plot(x,intensity)
plt.plot(x,intensity_single_order_y)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_cross_section', dpi=300, bbox_inches='tight')

intensity = nif_beam_spot_FWHM_xavier(X, Y, sup_gauss_dict)
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

## LMJ-A CPP, quadsplitting

In [ ]:
x_offset = 0.31
y_offset = 0.16

mesh_lim = 2.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(0.0+x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(0.0-x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(0.0+x_offset, y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(0.0-x_offset, y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(X+x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(X-x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(X+x_offset, Y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(X-x_offset, Y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

## LMJ-A CPP, quadsplitting, defocused 20mm

In [ ]:
# Mike Belyaev's method
icone = 0
c0 = np.array([1.0])     # Guess
c1 = np.array([0.600]) # Guess
c2 = np.array([0.400]) # Guess
c3 = np.array([2.7])

FWHMx = [0.600]
FWHMy = [0.400]
print(FWHMx[0] / np.cos(59*np.pi/180.0)) # Circular spot at 59degrees from LEH

sup_gauss_dict = super_gaussian_shape(c0[icone], c1[icone], c2[icone], c3[icone])
sup_gauss_dict["FWHMx"] = FWHMx[icone]
sup_gauss_dict["FWHMy"] = FWHMy[icone]

x_offset = 0.43
y_offset = 0.28

mesh_lim = 2.0
num_points = 1000
x = np.linspace(-mesh_lim, mesh_lim, num_points)
y = np.linspace(-mesh_lim, mesh_lim, num_points)
X, Y = np.meshgrid(x,y)

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(x+x_offset, 0.0+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(x-x_offset, 0.0+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(0.0+x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(0.0-x_offset, y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(0.0+x_offset, y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(0.0-x_offset, y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.plot(x,intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Intensity ($1/I_{max}$)")
plt.savefig('LMJ_spot_type_A_quad_splitting', dpi=300, bbox_inches='tight')

intensity_sup_pos1 = nif_beam_spot_FWHM_mike(X+x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos2 = nif_beam_spot_FWHM_mike(X-x_offset, Y-y_offset, sup_gauss_dict)
intensity_sup_pos3 = nif_beam_spot_FWHM_mike(X+x_offset, Y+y_offset, sup_gauss_dict)
intensity_sup_pos4 = nif_beam_spot_FWHM_mike(X-x_offset, Y+y_offset, sup_gauss_dict)
intensity = intensity_sup_pos1 + intensity_sup_pos2 + intensity_sup_pos3 + intensity_sup_pos4
plt.figure()
plt.pcolormesh(X, Y, intensity)
plt.xlabel("Width (mm)")
plt.ylabel("Length (mm)")
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('LMJ_spot_type_A_spot', dpi=300, bbox_inches='tight')

# Check Polar Coord Generation

In [ ]:
import healpy_pointings as hpoint

In [ ]:
nr = 10000

x = np.random.rand(nr) * 2.0 - 1.0
y = np.random.rand(nr) * 2.0 - 1.0
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nr))
cart_y_coords = np.zeros((nr))
polar_p_coords = np.zeros((nr))
polar_t_coords = np.zeros((nr))
for ir in range(nr):
    cart_x_coords[ir] = x[ir]
    cart_y_coords[ir] = y[ir]

    r, offset_phi = hpoint.square2disk(x[ir], y[ir])
    offset_theta = theta_max * r

    polar_p_coords[ir] = offset_phi
    polar_t_coords[ir] = offset_theta

X = cart_x_coords
Y = cart_y_coords

fig = plt.figure()
plt.plot(X,Y,"*")

print(np.min(polar_p_coords))
print(np.max(polar_p_coords))

U = np.degrees(polar_t_coords * np.cos(polar_p_coords))
V = np.degrees(polar_t_coords * np.sin(polar_p_coords))

fig1 = plt.figure()
plt.plot(U,V,"*")

In [ ]:
nx = 50
ny = 50

x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
theta_max = np.pi / 4.0

cart_x_coords = np.zeros((nx,ny))
cart_y_coords = np.zeros((nx,ny))
polar_p_coords = np.zeros((nx,ny))
polar_t_coords = np.zeros((nx,ny))
for ix in range(nx):
    for iy in range(ny):
        cart_x_coords[ix,iy] = x[ix]
        cart_y_coords[ix,iy] = y[iy]
        
        r, offset_phi = hpoint.square2disk(x[ix], y[iy])
        offset_theta = theta_max * r
        
        polar_p_coords[ix,iy] = offset_phi
        polar_t_coords[ix,iy] = offset_theta

X = cart_x_coords.flatten()
Y = cart_y_coords.flatten()

fig = plt.figure()
plt.plot(X,Y,"*")

phi = polar_p_coords.flatten()
theta = polar_t_coords.flatten()
U = np.degrees(theta * np.cos(phi))
V = np.degrees(theta * np.sin(phi))

fig1 = plt.figure()
plt.plot(U,V,"*")

# Neural Network

In [ ]:
import neural_network_generation as nng
num_epochs = 20
num_nn = 1
use_final_sigmoid = 1

In [ ]:
nn_params, nn_dataset, sys_params, nn_hyperparams = nng.main((_, run_dir, num_epochs, num_nn, use_final_sigmoid))

In [ ]:
fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')

## Compare NN

In [ ]:
import netcdf_read_write as nrw

In [ ]:
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(nn_hyperparams["initialize_seed"], nn_hyperparams["test_acc"], "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Random Seed')

## Compare NN accuracy from weights

In [ ]:
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import neural_network_generation as nng

In [ ]:
num_nn = 1

In [ ]:
initialize_seed = np.arange(num_nn)
test_acc = np.zeros(num_nn)

sys_params = tdg.define_system_params(run_dir)
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

nn_params = nng.define_nn_params(num_nn)
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

for inn in range(num_nn):
    nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
    nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
    nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
    y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
    test_acc[inn] = np.mean(np.abs(nn_dataset["Y_test"] - y_pred.T))

fig = plt.figure()
ax1 = plt.axes()
ax1.plot(initialize_seed, test_acc, "x")
ax1.set_ylabel('Absolute Test Accuracy')
ax1.set_xlabel('Neural Network Random Seed')
plt.savefig(sys_params["figure_location"]+"/Compare_NN" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Run Prediction Through Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10

In [ ]:
run_dir = "Data"
test_pred_dir = "Data_test_pred_1"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params(run_dir)
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)

nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
LMAX = dataset_params["LMAX"]
dataset_params["Y_train"] = nn_dataset["Y_train"].T
dataset_params["X_train"] = nn_dataset["X_train"].T
ifriit_inputs_true = Y_all
modes_true_all = X_all

# Change the number of modes to LMAX
modes_true = nng.change_number_modes(modes_true_all, avg_powers_all, LMAX)

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
ifriit_inputs_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
dataset_params["Y_train"] = ifriit_inputs_pred

mean_true_val = np.mean(ifriit_inputs_true)
print("The absolute error is: ", np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred)))
print("The percentage error relative to the mean is: ", np.mean((np.abs(ifriit_inputs_true - ifriit_inputs_pred))) / mean_true_val * 100, "%")

abs_error = np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred), axis=1)
rel_error = np.mean(np.abs(ifriit_inputs_true - ifriit_inputs_pred)/ ifriit_inputs_true , axis=1) * 100
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

for icone in range(4):
    print("Cone ", icone+1, " error: {:.2f}%".format(np.sum(abs_error[slice(icone,16,4)]) / 4.0 * 100.0 / mean_true_val))

parameter_names = ["offset_theta", "offset_phi", "cone_defocus", "cone_power"]
for icone in range(4):
    print(parameter_names[icone], " error: {:.2f}%".format(np.sum(abs_error[icone*4:(icone+1)*4]) / 4.0 * 100.0 / mean_true_val))

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
sys_params = tdg.define_system_params(test_pred_dir)
num_nn = 1
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
modes_pred = uim.change_number_modes(X_all, avg_powers_all, LMAX)

mean_val_true = np.mean(np.abs(modes_true))
print("The absolute error is: ", np.mean(np.abs(modes_true - modes_pred)))
print("Guessing zero for all modes gives error: ", np.mean(np.abs(modes_true)))
print("The percentage relative error is: ", np.mean(np.abs(modes_true - modes_pred) / modes_true) * 100.0, "%")

abs_error = np.mean(np.abs(modes_true - modes_pred), axis=1)
rel_error = np.mean(np.abs(modes_true - modes_pred) / modes_true, axis=1) * 100.0
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

fig = plt.figure()
ax = plt.axes()
plt.plot(abs_error)
plt.xlabel("Mode")
plt.ylabel("Average Absolute Error")
plt.savefig(sys_params["figure_location"]+'/absolute_error_averaged_'+ str(nn_params["num_examples"]) +"ex" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Automate Finding Minima

In [ ]:
test_pred_dir = "Data_minima3"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params("Data_220713_100ex")
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)
nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)

nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])
#print(nn_dataset["X_train"][1,0], nn_hyperparams["mu"][inn]/nn_hyperparams["sigma"][inn], nn_hyperparams["mu"][inn]-nn_hyperparams["sigma"][inn])
#print(np.shape(nn_dataset["X_train"]))
#print(nn_dataset["X_train"][0,:])
new_min = nn_hyperparams["mu"][inn]/nn_hyperparams["sigma"][inn]
nn_dataset["X_train"] = - new_min + new_min * np.random.rand(np.shape(nn_dataset["X_train"])[0], np.shape(nn_dataset["X_train"])[1])
#print(np.shape(nn_dataset["X_train"]))
#print(nn_dataset["X_train"][0,:])
dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
y_true = dataset_params["Y_train"]

# apply neural net to test set
sys_params = tdg.define_system_params(test_pred_dir)
y_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
iex = 0

num_examples = 1
sys_params = tdg.define_system_params(test_pred_dir)
sys_params["run_gen_deck"] = False
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
dataset_params["Y_train"] = Y_all[:,iex]
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)

mean_power_fraction = uim.extract_run_parameters(dataset_params, facility_spec, sys_params)

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"], run_data['target_radius'])
intensity_map_rms_spatial = uim.readout_intensity(facility_spec, intensity_map, mean_power_fraction)


# convert from W/sr to W/cm**2
intensity_map = intensity_map * (facility_spec['target_radius'] / 10000.0)**2
hp.mollview(intensity_map,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()
port_theta = facility_spec["Port_centre_theta"]
port_phi = facility_spec["Port_centre_phi"]
hp.projscatter(port_theta, port_phi)
hp.projscatter(np.squeeze(dataset_params["theta_pointings"]), np.squeeze(dataset_params["phi_pointings"]))
plt.savefig(sys_params["figure_location"]+'/intensity_mollweide_'+ str(iex) + "low_rms" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+ str(iex) +"low_rms" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Run PDD through NN

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
import utils_intensity_map as uim
%matplotlib inline
num_epochs = 10
num_examples = 1
nif_pdd_dir = "Data_pdd"
nif_pdd_pred_dir = "Data_pdd_pred"

In [ ]:
imap_nside = 256
LMAX = 30
input_size = 992
output_size = 16
target_radius_microns = 1100

run_location = nif_pdd_dir + "/run_0"
intensity_map = nrw.read_intensity(run_location, imap_nside, target_radius_microns)
X_train, avg_powers_all = uim.create_xtrain(intensity_map, LMAX)
X_all = np.reshape(X_train, (-1,input_size))
Y_all = np.zeros(output_size)

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights("neural_network_weights" + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# param setup
sys_params = tdg.define_system_params(nif_pdd_pred_dir)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
nn_dataset = nng.normalise(nn_dataset, mean=nn_hyperparams["mu"][inn], std_dev=nn_hyperparams["sigma"][inn])

y_pred = tfnn.apply_network(nn_dataset["X_train"].T, nn_weights, nn_hyperparams["use_final_sigmoid"][inn])

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = y_pred

In [ ]:
# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

# NN as surrogate for Ifriit

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import tf_neural_network as tfnn
import netcdf_read_write as nrw
import utils_deck_generation as idg
import training_data_generation as tdg
import utils_intensity_map as uim
import neural_network_generation as nng
%matplotlib inline
num_epochs = 10
num_nn = 1
run_dir = "Data"
imap_nside = 256
LMAX = 30
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"
use_final_sigmoid = 0

In [ ]:
# choose test data set
sys_params = tdg.define_system_params(run_dir)
nn_params = nng.define_nn_params(num_nn)

X_all, Y_all, avg_powers_all = nrw.import_training_data_reversed(sys_params, LMAX)
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata 
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)

In [ ]:
sys_params = tdg.define_system_params(run_dir)
sys_params["trainingdata_filename"] = filename_flipped_trainingdata
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

In [ ]:
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params)
#nn_dataset = nng.normalise(nn_dataset)
nn_hyperparams = nng.define_nn_hyperparams(num_epochs, num_nn, use_final_sigmoid=use_final_sigmoid)
nn_hyperparams = nng.multiple_nn(nn_params, nn_dataset, sys_params, nn_hyperparams)

## Check performance on test set

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(nn_dataset["X_test"], nn_weights, use_final_sigmoid)

In [ ]:
iex = 0

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), nn_dataset["Y_test"][iex,:] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"original" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,iex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Test on Independent Set

In [ ]:
run_dir = "Data_220713_100ex"
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

# choose neural network
inn = 0
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
nn_weights = nrw.read_nn_weights(nn_params["dir_nn_weights"] + "/" + "NN" + str(inn))
filename_hyperparams = nn_params["dir_nn_weights"] + "/" + nn_params["filename_hyperparams"]
nn_hyperparams = nrw.read_nn_weights(filename_hyperparams)

# choose test data set
sys_params = tdg.define_system_params(run_dir)
sys_params["trainingdata_filename"] = filename_flipped_trainingdata
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

nn_params = nng.define_nn_params(num_nn)
nn_params["use_test_set"] = False
nn_dataset = nng.seperate_test_set(X_all, Y_all, avg_powers_all, nn_params, use_test_set=False)

dataset_params, facility_spec = tdg.define_dataset_params(nn_params["num_examples"])
dataset_params["Y_train"] = nn_dataset["Y_train"].T
dataset_params["X_train"] = nn_dataset["X_train"].T
ifriit_inputs_true = dataset_params["X_train"]
modes_true = dataset_params["Y_train"]

# apply neural net to test set
modes_pred = tfnn.apply_network(nn_dataset["X_train"], nn_weights, nn_hyperparams["use_final_sigmoid"][inn])

In [ ]:
mean_val_true = np.mean(np.abs(modes_true))
print("The absolute error is: ", np.mean(np.abs(modes_true - modes_pred)))
print("Guessing zero for all modes gives error: ", np.mean(np.abs(modes_true)))
print("The percentage relative error is: ", np.mean(np.abs(modes_true - modes_pred) / modes_true) * 100.0, "%")

abs_error = np.mean(np.abs(modes_true - modes_pred), axis=1)
rel_error = np.mean(np.abs(modes_true - modes_pred) / modes_true, axis=1) * 100.0
print(np.shape(abs_error))
print(abs_error)
print(rel_error)

fig = plt.figure()
ax = plt.axes()
plt.plot(abs_error)
plt.xlabel("Mode")
plt.ylabel("Average Absolute Error")
plt.savefig(sys_params["figure_location"]+'/absolute_error_averaged_'+ str(nn_params["num_examples"]) + "ex" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

## Find Minimum in Train set

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
y_pred = tfnn.apply_network(X_all.T, nn_weights, use_final_sigmoid)

In [ ]:
mindex = np.argmin(np.mean(Y_all, axis=0))
print(mindex, " out of ", np.shape(Y_all))
mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
print(mindex, " out of ", np.shape(Y_all))
print(np.sqrt(np.sum(Y_all[:,mindex]**2, axis=0)))
print(np.shape(Y_all[:,mindex]), X_all[:,mindex])

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), Y_all[:,mindex] * 100.0)
#plt.plot(np.arange(LMAX), Y_all[:,iex])
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, Ifriit Inverse Ray Trace")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"original" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,mindex] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate Model")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(mindex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
num_examples = 10
run_dir = "Data_min10"

#min_slice = np.argsort(np.mean(Y_all, axis=0))[:num_examples]
min_slice = np.argsort(np.sqrt(np.sum(Y_all**2, axis=0)))[:num_examples]
print(min_slice)

dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
dataset_params["Y_train"] = X_all[:,min_slice]

In [ ]:
sys_params = tdg.define_system_params(run_dir)

# Create new run files
#sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
#facility_spec['target_radius'] = 1053.0
#facility_spec['default_power'] = 2.5

In [ ]:
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

## Search for Minima

In [ ]:
num_nn = 1
nn_params = nng.define_nn_params(num_nn)
x_test_random = np.random.rand(1000000,16)
print(x_test_random[0,:])

In [ ]:
use_final_sigmoid = 0
filename_hyperparams = nn_params["dir_nn_weights"] + "/NN0"
nn_weights = nrw.read_nn_weights(filename_hyperparams)
#y_pred = tfnn.apply_network(X_all.T, nn_weights, use_final_sigmoid)
y_pred = tfnn.apply_network(x_test_random, nn_weights, use_final_sigmoid)

In [ ]:
num_examples = 10
iex = 0
mindex = np.argmin(np.mean(y_pred, axis=0))
min_slice = np.argsort(np.mean(y_pred, axis=0))[:num_examples]
print(mindex)
print(np.shape(x_test_random[min_slice[:],:].T))

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[iex]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
dataset_params, facility_spec = tdg.define_dataset_params(num_examples)
#print(np.shape(dataset_params["Y_train"]))
#print(np.shape(x_test_random[min_slice,:]))
dataset_params["Y_train"] = x_test_random[min_slice[:],:].T

run_dir = "Data_nn_find_min"
sys_params = tdg.define_system_params(run_dir)

# Create new run files
sys_params["run_clean"] = False
dataset_params = idg.create_run_files(dataset_params, sys_params, facility_spec)
tdg.generate_training_data(dataset_params, sys_params, facility_spec)

In [ ]:
# choose test data set
sys_params = tdg.define_system_params(run_dir)

X_all, Y_all, avg_powers_all = nrw.import_training_data_reversed(sys_params, LMAX)
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata 
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
dataset_params, facility_spec = tdg.define_dataset_params(np.shape(X_all)[1])

#mindex = np.argmin(np.mean(Y_all, axis=0))
mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
print(mindex)

In [ ]:
iex = 0
with_pointing_markers = True

print(dataset_params[\"Y_train\"][iex])

num_examples = 1
sys_params = tdg.define_system_params(run_dir)
sys_params["run_gen_deck"] = False
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))

run_location = sys_params["root_dir"] + "/" + sys_params["sim_dir"] + str(iex)
intensity_map = nrw.read_intensity(run_location, dataset_params["imap_nside"], facility_spec['target_radius'])

In [ ]:
LMAX = dataset_params["LMAX"]
power_spectrum_unweighted, power_spectrum_weighted = uim.power_spectrum(intensity_map, LMAX)
x_max = LMAX

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), y_pred[:,min_slice[iex]] * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.title("Unweighted Modes, NN Surrogate find Minimum")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"nn_min_guess" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/unweighted_modes_"+str(iex)+"ifriit" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted * 100.0)
ax.set_xticks(range(0, LMAX+1, int(LMAX/5)))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)")
plt.savefig(sys_params["figure_location"]+"/weighted_modes" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')